In [2]:
# Data Manipulation
# ======================================================
import pandas as pd 
import numpy as np
from os import path
import configparser
from itertools import *
import math
from epiweeks import Week
from datetime import date, datetime
from calendar import month_name, month_abbr
from sklearn import preprocessing
from helper_functions import *

# Warnings Config
# ======================================================
import warnings
warnings.filterwarnings('ignore')

# Modeling & Forecasting
# ======================================================
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg, ar_select_order
from statsmodels.regression.linear_model import OLS
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.metrics import mean_squared_error

from lightgbm import LGBMRegressor
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, FunctionTransformer, PolynomialFeatures
from sklearn.feature_selection import RFECV
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer, make_column_selector

from skforecast.ForecasterBaseline import ForecasterEquivalentDate
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect

from skforecast.model_selection import bayesian_search_forecaster
from skforecast.model_selection import backtesting_forecaster

# Plotting
# ======================================================
import plotly.express as px
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [30]:
df = pd.read_csv('data.csv', index_col='weekstart')

In [31]:
df.index = pd.to_datetime(df.index)

In [33]:
# Resampling data so pandas understands that it's weekly
df = df.reset_index().set_index('weekstart').resample('W').first().fillna(method='bfill')

In [34]:
end_train = date(2015,12,31)
end_validation = date(2017,12,31)

df_train = df.loc[:end_train,:]
df_val = df.loc[end_train:end_validation,:]
df_test = df.loc[end_validation:,:]

print(f"Dates train      : {df_train.index.min()} --- {df_train.index.max()}  (n={len(df_train)})")
print(f"Dates validacion : {df_val.index.min()} --- {df_val.index.max()}  (n={len(df_val)})")
print(f"Dates test       : {df_test.index.min()} --- {df_test.index.max()}  (n={len(df_test)})")

Dates train      : 2005-01-09 00:00:00 --- 2015-12-27 00:00:00  (n=573)
Dates validacion : 2016-01-03 00:00:00 --- 2017-12-31 00:00:00  (n=105)
Dates test       : 2017-12-31 00:00:00 --- 2019-12-29 00:00:00  (n=105)


In [35]:
# Create and train RandomForest forecaster
# ==============================================================================
forecaster_rf1 = ForecasterAutoreg(
                regressor = RandomForestRegressor(random_state=123),
                lags      = 52
            )

forecaster_rf1.fit(y=df_train['cases'])
forecaster_rf1

ForecasterAutoreg 
Regressor: RandomForestRegressor(random_state=123) 
Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52] 
Transformer for y: None 
Transformer for exog: None 
Window size: 52 
Weight function included: False 
Differentiation order: None 
Exogenous included: False 
Type of exogenous variable: None 
Exogenous variables names: None 
Training range: [Timestamp('2005-01-09 00:00:00'), Timestamp('2015-12-27 00:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: W-SUN 
Regressor parameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 123, 'verbose': 0, 'war

In [36]:
# Backtest model on test data
# ==============================================================================
metric_rf1, predictions_rf1 = backtesting_forecaster(
                        forecaster         = forecaster_rf1,
                        y                  = df.loc[:end_validation]['cases'],
                        steps              = 6,
                        metric             = 'mean_absolute_error',
                        initial_train_size = len(df[:end_train]),
                        refit              = False,
                        n_jobs             = 'auto',
                        verbose            = False,
                        show_progress      = True
                    )

print(f'Backtest error (MAE): {metric_rf1}')

100%|██████████| 18/18 [00:00<00:00, 39.29it/s]

Backtest error (MAE): 177.68809523809523


In [24]:
# Hyperparameters search
# ==============================================================================
# Lags grid
lags_grid = [[52],[1],[1,52],[1,2],[1,2,52],[1,2,3],[1,2,3,52]]

# Regressor hyperparameters search space
def search_space(trial):
    search_space  = {
        'n_estimators'  : trial.suggest_int('n_estimators', 400, 1200, step=100),
        'max_depth'     : trial.suggest_int('max_depth', 3, 10, step=1),
    } 
    return search_space

results_search_rf1, frozen_trial_rf1 = bayesian_search_forecaster(
                                forecaster         = forecaster_rf1,
                                y                  = df.loc[:end_validation, 'cases'], # Test data not used
                                search_space       = search_space,
                                lags_grid          = lags_grid,
                                steps              = 6,
                                refit              = False,
                                metric             = 'mean_absolute_error',
                                initial_train_size = len(df_train),
                                fixed_train_size   = False,
                                n_trials           = 20, # Increase this value for a more exhaustive search
                                random_state       = 123,
                                return_best        = True,
                                n_jobs             = 'auto',
                                verbose            = False,
                                show_progress      = True
                            )

Number of models compared: 140,
         20 bayesian search in each lag configuration.


lags grid: 100%|██████████| 7/7 [24:22<00:00, 208.98s/it]


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2 52] 
  Parameters: {'n_estimators': 400, 'max_depth': 10}
  Backtesting metric: 210.1131382519461



In [28]:
results_search_rf1.head()

,lags,params,mean_absolute_error,n_estimators,max_depth
90,"[1, 2, 52]","{'n_estimators': 400, 'max_depth': 10}",210.113138,400,10
91,"[1, 2, 52]","{'n_estimators': 400, 'max_depth': 10}",210.113138,400,10
92,"[1, 2, 52]","{'n_estimators': 400, 'max_depth': 10}",210.113138,400,10
93,"[1, 2, 52]","{'n_estimators': 400, 'max_depth': 10}",210.113138,400,10
137,"[1, 2, 3, 52]","{'n_estimators': 1200, 'max_depth': 7}",213.432008,1200,7


In [37]:
# Create and train RandomForest forecaster
# ==============================================================================
forecaster_rf2 = ForecasterAutoreg(
                regressor = RandomForestRegressor(random_state=123,
                                                    n_estimators=400,
                                                    max_depth=10),
                lags      = [1,2,52]
            )

forecaster_rf2.fit(y=df_train['cases'])
forecaster_rf2

# Backtest model on test data
# ==============================================================================
metric_rf2, predictions_rf2 = backtesting_forecaster(
                        forecaster         = forecaster_rf2,
                        y                  = df.loc[:end_validation]['cases'],
                        steps              = 6,
                        metric             = 'mean_absolute_error',
                        initial_train_size = len(df[:end_train]),
                        refit              = False,
                        n_jobs             = 'auto',
                        verbose            = False,
                        show_progress      = True
                    )
print(f'Backtest error (MAE): {metric_rf2}')

100%|██████████| 18/18 [00:01<00:00, 11.13it/s]

Backtest error (MAE): 210.1131382519461


---

In [38]:
# Create and train RandomForest forecaster
# ==============================================================================
forecaster_rf3 = ForecasterAutoreg(
                regressor = RandomForestRegressor(random_state=123),
                lags      = 52
            )

forecaster_rf3.fit(y=df_train['cases'], exog=df_train.drop(columns=['cases']))
forecaster_rf3

ForecasterAutoreg 
Regressor: RandomForestRegressor(random_state=123) 
Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52] 
Transformer for y: None 
Transformer for exog: None 
Window size: 52 
Weight function included: False 
Differentiation order: None 
Exogenous included: True 
Type of exogenous variable: <class 'pandas.core.frame.DataFrame'> 
Exogenous variables names: ['epiweek', 'GS_cold', 'GS_cough', 'GS_fever', 'GS_flu', 'AWND', 'PRCP', 'SNOW', 'TAVG', 'TMAX', 'TMIN', 'Overall AQI Value', 'CO', 'Ozone', 'PM10', 'PM25', 'Days Good', 'Days Moderate', 'Days Unhealthy', 'visits', 'Main Pollutant_CO', 'Main Pollutant_NO2', 'Main Pollutant_Ozone', 'Main Pollutant_PM2.5', 'epiweek_sin', 'epiweek_cos', 'GS_cold_L1', 'GS_cold_L2', 'GS_fever_L1', 'GS_fever_L2', 'AWND_L1', 'TAVG_L1', 'TAVG_L2', 'TMAX_L1', 'TMAX_L2', 'TMIN_L1', 'TMIN_L2'] 
Training range: [Timestamp('2005-01-09

In [53]:
# Backtest model on test data
# ==============================================================================
metric_rf3, predictions_rf3 = backtesting_forecaster(
                        forecaster         = forecaster_rf3,
                        y                  = df.loc[:end_validation]['cases'],
                        exog               = df.loc[:end_validation].drop(columns=['cases']),
                        steps              = 6,
                        metric             = 'mean_absolute_error',
                        initial_train_size = len(df[:end_train]),
                        refit              = False,
                        n_jobs             = 'auto',
                        verbose            = False,
                        show_progress      = True
                    )

print(f'Backtest error (MAE): {metric_rf3}')

100%|██████████| 18/18 [00:00<00:00, 37.08it/s]

Backtest error (MAE): 180.50485714285716


In [39]:
# Create and train RandomForest forecaster
# ==============================================================================
forecaster_rf4 = ForecasterAutoreg(
                regressor = RandomForestRegressor(random_state=123,
                                                    n_estimators=400,
                                                    max_depth=10),
                lags      = [1,2,52]
            )

forecaster_rf4.fit(y=df_train['cases'], exog=df_train.drop(columns=['cases']))
forecaster_rf4

# Backtest model on test data
# ==============================================================================
metric_rf4, predictions_rf4 = backtesting_forecaster(
                        forecaster         = forecaster_rf4,
                        y                  = df.loc[:end_validation]['cases'],
                        exog               = df.loc[:end_validation].drop(columns=['cases']),
                        steps              = 6,
                        metric             = 'mean_absolute_error',
                        initial_train_size = len(df[:end_train]),
                        refit              = False,
                        n_jobs             = 'auto',
                        verbose            = False,
                        show_progress      = True
                    )

print(f'Backtest error (MAE): {metric_rf4}')

100%|██████████| 18/18 [00:02<00:00,  8.85it/s]

Backtest error (MAE): 203.45640612497138


In [40]:
forecaster_rf3.get_feature_importances().sort_values(by='importance', ascending=False)

,feature,importance
51,lag_52,0.252143
56,GS_flu,0.168111
0,lag_1,0.089475
60,TAVG,0.055130
71,visits,0.036157
...,...,...
74,Main Pollutant_Ozone,0.000290
75,Main Pollutant_PM2.5,0.000249
73,Main Pollutant_NO2,0.000073
70,Days Unhealthy,0.000016


In [41]:
# Underlying matrices used by the forecaster to fit the internal regressor
# ==============================================================================
X_train, y_train = forecaster_rf3.create_train_X_y(
                    y    = df_train['cases'],
                    exog = df_train.drop(columns=['cases'])
                )

# Recursive feature elimination with cross-validation
# ==============================================================================
# Sample 50% of the data to speed up the calculation
rng = np.random.default_rng(seed=785412)
sample = rng.choice(X_train.index, size=int(len(X_train)*0.5), replace=False)
X_train_sample = X_train.loc[sample, :]
y_train_sample = y_train.loc[sample]

regressor = RandomForestRegressor(random_state=123)

selector = RFECV(
    estimator              = regressor,
    step                   = 1,
    cv                     = 3,
    min_features_to_select = 15,
    n_jobs                 = -1
)
selector.fit(X_train_sample, y_train_sample)
selected_features_rfe = selector.get_feature_names_out()

print("Recursive feature elimination")
print("-----------------------------")
print(f"Total number of features available: {X_train.shape[1]}") 
print(f"Total number of records available: {X_train.shape[0]}")
print(f"Total number of records used for feature selection: {X_train_sample.shape[0]}")
print(f"Number of features selected: {len(selected_features_rfe)}")
print(f"Selected features : \n {selected_features_rfe}")

Recursive feature elimination
-----------------------------
Total number of features available: 89
Total number of records available: 521
Total number of records used for feature selection: 260
Number of features selected: 18
Selected features : 
 ['lag_1' 'lag_2' 'lag_3' 'lag_24' 'lag_29' 'lag_30' 'lag_33' 'lag_37'
 'lag_51' 'lag_52' 'epiweek' 'GS_flu' 'TAVG' 'TMIN' 'visits' 'AWND_L1'
 'TAVG_L1' 'TAVG_L2']


In [51]:
# Create forecaster with the selected features
# ==============================================================================

selected_exog_features = [
    feature
    for feature in selected_features_rfe
    if not feature.startswith("lag_")
]

forecaster_rfbest = ForecasterAutoreg(
                regressor        = RandomForestRegressor(random_state=123, 
                                        ),
                lags             = [1,2,3,24,29,30,33,37,51,52]
            )

forecaster_rfbest.fit(y=df_train['cases'], exog=df_train[selected_exog_features])

# Backtesting model with exogenous variables on test data
# ==============================================================================
metric_rfbest, predictions_rfbest = backtesting_forecaster(
                            forecaster         = forecaster_rfbest,
                            y                  = df[:end_validation]['cases'],
                            exog               = df[:end_validation][selected_exog_features],
                            steps              = 6,
                            metric             = 'mean_absolute_error',
                            initial_train_size = len(df[:end_train]),
                            refit              = False,
                            n_jobs             = 'auto',
                            verbose            = False,
                            show_progress      = True
                        )

print(f"Backtest error: {metric_rfbest:.2f}")

100%|██████████| 18/18 [00:00<00:00, 37.74it/s]

Backtest error: 181.08


In [56]:
# Create forecaster with the selected features
# ==============================================================================

selected_exog_features = [
    feature
    for feature in selected_features_rfe
    if not feature.startswith("lag_")
]

forecaster_rfbest = ForecasterAutoreg(
                regressor        = RandomForestRegressor(random_state=123, 
                                        ),
                lags             = 52
            )

forecaster_rfbest.fit(y=df_train['cases'], exog=df_train[selected_exog_features])

# Backtesting model with exogenous variables on test data
# ==============================================================================
metric_rfbest, predictions_rfbest = backtesting_forecaster(
                            forecaster         = forecaster_rfbest,
                            y                  = df[:end_validation]['cases'],
                            exog               = df[:end_validation][selected_exog_features],
                            steps              = 6,
                            metric             = 'mean_absolute_error',
                            initial_train_size = len(df[:end_train]),
                            refit              = False,
                            n_jobs             = 'auto',
                            verbose            = False,
                            show_progress      = True
                        )

print(f"Backtest error: {metric_rfbest:.2f}")

100%|██████████| 18/18 [00:00<00:00, 35.53it/s]

Backtest error: 181.48


In [55]:
print(f'Backtest error (MAE): {metric_rf1}')
print(f'Backtest error (MAE): {metric_rf2}')
print(f'Backtest error (MAE): {metric_rf3}')
print(f'Backtest error (MAE): {metric_rf4}')

Backtest error (MAE): 177.68809523809523
Backtest error (MAE): 210.1131382519461
Backtest error (MAE): 180.50485714285716
Backtest error (MAE): 203.45640612497138


In [54]:
print(f'RandomForest original MAE                :  {round(metric_rf1)}\n\
RandomForest best from gridsearch MAE    :  {round(metric_rf2)}\n\
RandomForest + exog MAE                  :  {round(metric_rf4)}')

RandomForest original MAE                :  178
RandomForest best from gridsearch MAE    :  210
RandomForest + exog MAE                  :  203
